In [1]:
import pandas as pd 
import datetime 
#通过odps接口直接取数
from odps import ODPS

In [2]:
# 每周六需要跑如下数据 
def get_yesterday():
    today = datetime.date.today()  # 获取当前日期  周六
    oneday = datetime.timedelta(days=1)  
    sevenday = datetime.timedelta(days=7)
    end_day = str(today - oneday).replace('-', '')  # 当前日期减去完整的一天
    start_day = str(today - sevenday).replace('-', '')
    return start_day, end_day
start_day, end_day = get_yesterday()

In [3]:
odps = ODPS('zrKChXTcUFNThEdl', 'u253aSzEVmAxJ0lsASQOkpPNRbaFXY', 'wywk_dt_dev',endpoint='http://service.cn.maxcompute.aliyun.com/api')


In [36]:
# 日新用户数  日美团新用户数  日老带新用户数  
get_new_user_sql = """
select t1.dt_day, COUNT(distinct t1.wyuid) cnt
       , count(distinct case when t2.source_channel = '美团' then t1.wyuid end) as mt_cnt
       , count(distinct case when t2.source_channel = '线上老带新' then t1.wyuid end) as intr_cnt
from  wywk_dt_base.bi_cust_account_day t1 
left join (select wyuid, dt_hour
                  ,CASE WHEN t6.if_import = 1 THEN '导入'
                        WHEN t6.source_meituan='1' THEN '美团'
                        WHEN t6.source_pos_laodaixin='1' THEN '线下老带新'
                        when t6.source_weixin_laodaixin='1' THEN '线上老带新'
                        when t6.source_koubei='1' THEN  '口碑'
                        WHEN t6.source_tianmao='1' THEN '天猫'
                        WHEN t6.source_tiktok='1' THEN '抖音'
                        ELSE '自然流量'
                   END as source_channel
           from   wywk_dt_base.bi_new_cust_hour t6 
           where  dt_hour >= '%s'
          ) t2 
       on t1.wyuid = t2.wyuid and t1.dt_day = t2.dt_hour
left join (select dt_day, wyid
           from   wywk_dt_base.bi_order_aggregate_marketing_detail t1
           where  t1.order_status = '1' -- 订单状态 1表示正常
             and  t1.EXCEPTION = 0  -- 是否退单 
             and  dt_day >= '%s'
           group by dt_day, wyid
          ) t3 on t1.dt_day = t3.dt_day and t1.wyuid = t3.wyid
where t1.dt_day >= '%s' and t1.account_type = 1
  and SUBSTR(GREATEST(t1.create_time_account, t1.temp_to_vip_time),1,10) 
          = substr(to_date(t1.dt_day, 'yyyymmdd'), 1, 10)
  and t2.source_channel <> '导入'  -- 非导入 
  and t3.wyid is not null -- 有任意订单 
group by t1.dt_day
order by t1.dt_day asc
;
"""%(start_day, start_day, start_day)

with odps.execute_sql(get_new_user_sql).open_reader() as reader:
    d = [] 
    for record in reader: 
        d.append(dict(record)) 
pd.DataFrame(d)

,dt_day,cnt,mt_cnt,intr_cnt
0,20220816,5776,2424,104
1,20220817,5619,2375,114
2,20220818,5688,2327,119
3,20220819,6584,2743,119
4,20220820,8961,3664,178
5,20220821,7804,3048,165
6,20220822,5953,2296,103


In [37]:
# 日营业门店数
get_open_shop_amount_sql = """
select dt_day, count(1) shop_cnt
from   (select dt_day, common_code
        from   wywk_dt_base.bi_order_aggregate_marketing_detail t1
        where  t1.order_status = '1' -- 订单状态 1表示正常
          and  t1.EXCEPTION = 0  -- 是否退单 
          and  dt_day >= '%s'
          and  common_code not in ('9991', '9992', '9989') 
          and  not(common_code like '%%DJ%%')
        group by dt_day, common_code
        ) t1 
group by dt_day
order by dt_day asc 
;
"""%(start_day)
with odps.execute_sql(get_open_shop_amount_sql).open_reader() as reader:
    d = [] 
    for record in reader: 
        d.append(dict(record)) 
pd.DataFrame(d)

,dt_day,shop_cnt
0,20220816,618
1,20220817,629
2,20220818,650
3,20220819,641
4,20220820,637
5,20220821,641
6,20220822,645


In [38]:
# 日订座率 

get_order_seat_sql = """
SELECT  date_shop
        , sum(case when is_holdseat='开通订座' then (座位数-订座失败座位数) else 0 end ) seat_cnt 
        , sum(case when is_holdseat='开通订座' and swrc > 0 then swrc else 0 end ) swrc
FROM    wywk_dt_dev.holdseat_zhuanxiang
WHERE   dt_day IS NOT NULL
  and   date_shop >= '%s'
group by date_shop
order by date_shop asc 
;
"""%(start_day) 
with odps.execute_sql(get_order_seat_sql).open_reader() as reader:
    d = [] 
    for record in reader: 
        d.append(dict(record)) 
pd.DataFrame(d)

,date_shop,seat_cnt,swrc
0,20220816,6018,54535
1,20220817,6070,54058
2,20220821,9235,68233
3,20220818,6092,54573
4,20220819,7816,57418
5,20220822,6698,56164
6,20220820,9959,65333


In [4]:
# 日线上化率 
start_day = '20220819'
get_online_rate_sql = """
select login_day, count(distinct online_cert) / count(distinct user_id) rate
from   wywk_dt_base.lzc_login_online_member 
where  dt_day >= '%s'
group by login_day
order by login_day asc 
;
"""%(start_day) 
with odps.execute_sql(get_online_rate_sql).open_reader() as reader:
    d = [] 
    for record in reader: 
        d.append(dict(record)) 
pd.DataFrame(d)

,login_day,rate
0,2022-08-19,0.430156
1,2022-08-22,0.436591
2,2022-08-26,0.414660
3,2022-08-27,0.403521
4,2022-08-21,0.450835
5,2022-08-28,0.383834
6,2022-08-20,0.448890
7,2022-08-23,0.431814
8,2022-08-24,0.436197
9,2022-08-25,0.414339


In [40]:
# 抖音粉丝数 订座页面访问率 - CDP 

In [41]:
# 营运周 门店渠道 新用户数 
get_yingyun_data_sql = """
select t1.dt_hour as `日期`, t2.shopname as `门店名`
       , count(case when t1.source_channel = '美团' then t1.wyuid end) as `美团`
       , count(case when t1.source_channel = '线上老带新' then t1.wyuid end) as `线上老带新`
       , count(case when t1.source_channel = '口碑' then t1.wyuid end) as `口碑`
       , count(case when t1.source_channel = '天猫' then t1.wyuid end) as `天猫`
       , count(case when t1.source_channel = '抖音' then t1.wyuid end) as `抖音`
       , count(case when t1.source_channel = '自然流量' then t1.wyuid end) as `自然流量`
from   (
        select t1.dt_hour, t1.wyuid, t1.source_channel 
               , case when t1.special_label = 1 then coalesce(t21.common_code, t3.common_code, '9991') else t1.commoncode_9991 end as shop
        from   (select wyuid, dt_hour, t1.commoncode_9991
                       , CASE WHEN t1.if_import = 1 THEN '导入'
                              WHEN t1.source_meituan='1' THEN '美团'
                            --   WHEN t1.source_pos_laodaixin='1' THEN '线下老带新'
                              when t1.source_weixin_laodaixin='1' THEN '线上老带新'
                              when t1.source_koubei='1' THEN  '口碑'
                              WHEN t1.source_tianmao='1' THEN '天猫'
                              WHEN t1.source_tiktok='1' THEN '抖音'
                              ELSE '自然流量'
                       END as source_channel
                       , case when commoncode_9991 in ('9991') or temp_to_vip_time is not null then 1 end special_label -- 临转会和门店是9991的用户标签
                from   wywk_dt_base.bi_new_cust_hour t1
                where  dt_hour >= '%s' 
                  and  t1.account_type = 1 
               ) t1 
        left join (select *
                          , row_number() over(partition by dt_day, wyid order by time asc) rn 
                   from   wywk_dt_base.bi_order_aggregate_marketing_detail t1
                   where  t1.order_status = '1' -- 订单状态 1表示正常
                   and  t1.EXCEPTION = 0  -- 是否退单 
                   and  dt_day >= '%s' 
                  ) t2 on t1.dt_hour = t2.dt_day and t1.wyuid = t2.wyid and t2.rn = 1 -- 当天有任意订单的用户 
        left join (select wyuid, dt_day, common_code, row_number() over(partition by wyuid, dt_day order by event_time asc) rn 
                   from   wywk_dt_base.net_track 
                   WHERE  event_type = 2 -- 上机 
                   AND dt_day >= '%s'
                  ) t21 -- 当天有上机记录的用户 
               on t1.dt_hour = t21.dt_day and t1.wyuid = t21.wyuid and t21.rn = 1 
        left join (select dt_day, wyid, common_code, row_number() over(partition by dt_day, wyid order by time asc) rn 
                   from   wywk_dt_base.bi_order_aggregate_marketing_detail t1
                   where  t1.order_status = '1' -- 订单状态 1表示正常
                   and  t1.EXCEPTION = 0  -- 是否退单 
                   and  dt_day >= '%s'
                   and  common_code not in ('9991', '9992', '9989') -- 线下店
                   and  not(common_code like '%%DJ%%')
                  ) t3 -- 当天在线下门店有任意订单的
               on t1.wyuid = t3.wyid and t1.dt_hour = t3.dt_day and t3.rn = 1 
        where t1.source_channel <> '导入'  -- 非导入 
          and t2.wyid is not null -- 有任意订单 
       ) t1 
left join wywk_dt_base.bi_shop_integrated_day t2 
       on t2.dt_day = '%s' and t1.shop = t2.commcode
group by t1.dt_hour, t2.shopname
;
"""%(start_day, start_day, start_day, start_day, end_day)
with odps.execute_sql(get_yingyun_data_sql).open_reader() as reader:
    d = [] 
    for record in reader: 
        d.append(dict(record)) 
df_yy = pd.DataFrame(d)
df_yy.to_excel('%s_营运周门店_渠道_新用户数.xlsx'%end_day) 